<a href="https://colab.research.google.com/github/vforjj/AI-ML-Review/blob/main/1%EC%83%9D%EB%AC%BC%EB%8B%A4%EC%96%91%EC%84%B1_%EC%9E%AC%EB%AC%B4%EA%B0%80%EC%B9%98_%ED%99%98%EC%82%B0_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
자연자본 리스크 인텔리전스 플랫폼 (Resilience Capital Platform)
[Gyeongbuk Special Edition]

경북 지역 대형 산불(99,289 ha, 1조 505억 피해) 데이터를 기반으로
생물다양성 복원(Reforestation) 시 기업의 TNFD 기여도 및 화폐 가치를 산출하는 모델.
"""

from typing import Dict, List, Optional

# ====================================================================
# 1. 경북 산불 실제 데이터 및 재무 프록시 설정 (User Data Inputs)
# ====================================================================

# 사용자 제공 통계 (중앙재난안전대책본부 등)
TOTAL_DAMAGE_KRW = 1_505_000_000_000  # 총 피해액 (1조 505억 원)
TOTAL_RECOVERY_COST_KRW = 1_831_000_000_000  # 총 복구비 (1조 8,310억 원)
TOTAL_AFFECTED_AREA_HA = 99_289.0  # 피해 면적 (ha)

# 단위 면적당 가치 도출 (Per Hectare Metrics)
DAMAGE_PER_HA = TOTAL_DAMAGE_KRW / TOTAL_AFFECTED_AREA_HA
RECOVERY_COST_PER_HA = TOTAL_RECOVERY_COST_KRW / TOTAL_AFFECTED_AREA_HA

# 생물다양성 관련 경제 지표 (Proxies)
BIODIVERSITY_PREMIUM_MULTIPLIER = 1.2
CARBON_PRICE_KRW = 45000
CARBON_SEQUESTRATION_RATE = 11.5

# 사회적 기여 지표
LOCAL_JOB_ANNUAL_INCOME_KRW = 32_000_000
LOCAL_ECONOMIC_MULTIPLIER = 1.5

# [기준점] 산림청 표준 조림 밀도 (Standard Planting Density per HA)
# 보통 1ha당 3,000그루를 표준으로 보며, 이 대비 현재 밀도를 가중치에 반영
STANDARD_DENSITY_PER_HA = 3000.0

# 종별 생태계 중요도 가중치 (Ecological Importance Weights)
SPECIES_WEIGHTS = {
    "산양(Amur Goral)": 100.0,      # 천연기념물: 발견 자체의 희소 가치
    "금강소나무_치수": 0.05,        # 개체당 가치는 낮으나(어린 나무), 밀도(Density)가 중요 기준점
    "멧돼지/고라니": 1.0,           # 일반 종: 기초 생태계 구성
    "까막딱따구리": 50.0,           # 멸종위기 2급: 산림 건강도 지표
    "하늘다람쥐": 30.0              # 멸종위기 2급: 성숙림 지표
}

# ====================================================================
# 2. 기업 TNFD 기여도 측정 클래스
# ====================================================================

class TNFDImpactCalculator:
    def __init__(self, company_investment_krw: float, restoration_period_years: int = 10):
        self.investment = company_investment_krw
        self.period = restoration_period_years
        self.restored_area_ha = (self.investment / RECOVERY_COST_PER_HA) * 1.1

    def simulate_biodiversity_recovery(self, current_month: str) -> Dict[str, float]:
        """
        2026년 3-4월 생물다양성 조사 데이터를 기반으로 회복률 시뮬레이션
        [업데이트] 산림 표준 밀도(Standard Density)를 기준점으로 적용
        """
        species_count = {}

        if current_month in ["3", "4", "March", "April"]:
            # 실제 데이터가 없는 경우를 대비한 가상 관측값 (AI/드론 조사 결과)
            species_count = {
                "산양(Amur Goral)": 2,
                "금강소나무_치수": 2500,    # 1ha당 2,500그루 관측 가정 (표준 3,000 대비 약 83%)
                "멧돼지/고라니": 5
            }
        else:
            species_count = {
                "산양(Amur Goral)": 0,
                "금강소나무_치수": 1200,
                "멧돼지/고라니": 2
            }

        # 가중치 기반 점수 계산
        total_score = 0.0
        for species, count in species_count.items():
            weight = SPECIES_WEIGHTS.get(species, 1.0)

            # [로직 추가] 금강소나무 치수의 경우 '표준 밀도' 대비 달성도를 가중치에 곱함
            if species == "금강소나무_치수":
                density_ratio = min(count / STANDARD_DENSITY_PER_HA, 1.0)
                # 밀도가 높을수록 산림 복원력 점수 가속화
                total_score += (count * weight) * (1 + density_ratio)
            else:
                total_score += count * weight

        # 회복률 환산 (샘플 지역 목표 점수 기준)
        target_sample_score = 500.0
        calculated_rate = min(total_score / target_sample_score, 1.0)

        # 산양 발견 시 최소 회복률 보정
        if species_count.get("산양(Amur Goral)", 0) > 0:
            calculated_rate = max(calculated_rate, 0.45)

        return {"recovery_rate": calculated_rate, "species_data": species_count, "total_score": total_score}

    def calculate_monetary_impact(self, recovery_rate: float, indigenous_jobs_count: int = 0) -> Dict[str, float]:
        """
        TNFD 공시용 기업 기여도 화폐 가치 산출 함수
        """
        # 1. 재해 예방 가치 (Avoided Risk)
        avoided_damage_value = self.restored_area_ha * DAMAGE_PER_HA * recovery_rate

        # 2. 탄소 흡수 가치 (Climate Mitigation)
        carbon_value = self.restored_area_ha * CARBON_SEQUESTRATION_RATE * self.period * CARBON_PRICE_KRW

        # 3. 생물다양성 자산 가치 (Nature Positive Asset)
        biodiversity_value = (self.investment * BIODIVERSITY_PREMIUM_MULTIPLIER) - self.investment
        if biodiversity_value < 0: biodiversity_value = 0

        # 4. 지역사회/원주민 일자리 창출 가치 (Social Livelihood Value)
        livelihood_value = indigenous_jobs_count * LOCAL_JOB_ANNUAL_INCOME_KRW * self.period * LOCAL_ECONOMIC_MULTIPLIER

        # 5. 총 사회적 기여 가치 (Total Societal Value)
        total_value = avoided_damage_value + carbon_value + biodiversity_value + livelihood_value

        # 6. ROI (사회적 투자 수익률)
        sroi = (total_value / self.investment) * 100 if self.investment > 0 else 0

        return {
            "Restored_Area_Ha": round(self.restored_area_ha, 2),
            "TNFD_Avoided_Risk_KRW": round(avoided_damage_value, 0),
            "TNFD_Carbon_Opportunity_KRW": round(carbon_value, 0),
            "TNFD_Biodiversity_Asset_KRW": round(biodiversity_value, 0),
            "TNFD_Social_Livelihood_KRW": round(livelihood_value, 0),
            "Total_Impact_Value_KRW": round(total_value, 0),
            "Social_ROI_Percent": round(sroi, 2)
        }

# ====================================================================
# 3. 실행 및 리포트 생성
# ====================================================================

if __name__ == "__main__":
    COMPANY_INVESTMENT = 10_000_000_000 # 100억 원
    TARGET_MONTH = "April"
    INDIGENOUS_JOBS = 50

    print("="*60)
    print(f"🌲 경북 산불 피해 복원: 기업 TNFD 기여도 측정 모델 🌲")
    print(f"투자 기업: A Corp | 투자 금액: {COMPANY_INVESTMENT/100000000:,.0f}억 원")
    print("-" * 60)

    calculator = TNFDImpactCalculator(COMPANY_INVESTMENT, restoration_period_years=10)
    bio_survey = calculator.simulate_biodiversity_recovery(TARGET_MONTH)
    recovery_rate = bio_survey['recovery_rate']

    print(f"[Step 1] 2026년 4월 생태 조사 결과")
    print(f"  - 금강소나무 치수 관측: {bio_survey['species_data']['금강소나무_치수']} 그루/ha")
    print(f"  - 산림 표준 대비 밀도도: {(bio_survey['species_data']['금강소나무_치수']/STANDARD_DENSITY_PER_HA)*100:.1f}%")
    print(f"  - 생태계 가중 점수 합계: {bio_survey['total_score']:.1f}점")
    print(f"  - 최종 산출된 회복 지수: {recovery_rate*100:.1f}%")

    impact_metrics = calculator.calculate_monetary_impact(recovery_rate, INDIGENOUS_JOBS)

    print(f"\n[Step 2] TNFD 공시용 화폐 가치")
    print(f"  ▶ 총 사회적 기여 가치 (Total Value): {impact_metrics['Total_Impact_Value_KRW']/100000000:,.2f}억 원")
    print(f"  ▶ 사회적 투자 수익률 (S-ROI): {impact_metrics['Social_ROI_Percent']:.1f}%")
    print("="*60)